In [11]:
import pandas as pd
import numpy as np
import gurobipy as gp
from gurobipy import GRB

## 使用Tuplelist与Tupledict的原因

list 用 [ ] 表示，适合做下标、索引、变量、约束等各种对象的**集合**

tuple 用 ( ) 表示，因其不可修改的特性，其适合做**多维下标**

dict 用 { } 表示，适合用来表示**带下标的数值**，例如变量

但是Python的 list/tuple/dict 均有不足，例如在建模过程中，经常需要对下标数据做挑选，不同下标的数据进行组合; 当使用上述三种方式时，智能全部循环+if条件进行筛选，效率非常低，所以需要采用Gurobi的扩展对象 TupleList与TupleDict

### Tuplelist

Tuplelist可以使用select函数去选择

在tupledict中可以使用 select / sum / prod 进行 选择 / 求和 / 求积之和

Gurobi的变量一般都是tupledict形式的，因为需要有一个下标去定义，且有sum/prod这样的函数

建模时建议采用tuplelists筛选和指定合适的下标组合，再基于这些组合关系建立变量和数据字典；然后利用gp.tupledict.select() / gp.tupledict.sum() / gp.tupledict.prod()来对下标进行组合处理 


In [90]:
# OD-Pairs
cities = [('A', 'B'), ('A', 'C'), ('B', 'C'), ('B', 'D'), ('C', 'D')]
routes = gp.tuplelist(cities)

现在需要选出所有O为'A'的OD-Pairs

In [6]:
# 正常使用list的方法

result = []
for i,j in cities:
    if i == 'A':
        result.append((i,j))
result

[('A', 'B'), ('A', 'C')]

In [46]:
# 使用Grurobi自带的tuplelist的select方法
# '*' 代表任意值
# [XXX] 代表[]内任何值均可匹配


print("select('A', '*'): ", '\n', routes.select('A', '*'))
print('\n')
print("select(['A', 'B'], '*'): ", '\n', routes.select(['A', 'B'], '*'))
print('\n')
print("select('A', 'C'): ", '\n', routes.select('A', 'C'))

select('A', '*'):  
 <gurobi.tuplelist (2 tuples, 2 values each):
 ( A , B )
 ( A , C )
>


select(['A', 'B'], '*'):  
 <gurobi.tuplelist (4 tuples, 2 values each):
 ( A , B )
 ( A , C )
 ( B , C )
 ( B , D )
>


select('A', 'C'):  
 <gurobi.tuplelist (1 tuples, 2 values each):
 ( A , C )
>


### Tupledict

tupledict 是一个key为tuple的dictinary

在tupledict中可以使用 select - 选择 / sum - 求和 / prod - 求积

因为数学规划的变量一般都需要有下标，且去求和或求积，所以Gurobi的变量一般都是tupledict形式

建模时建议采用tuplelist筛选和指定合适的下标组合，再基于这些组合关系建立变量和数据字典；然后利用gp.tupledict.select() / gp.tupledict.sum() / gp.tupledict.prod()来对下标进行组合处理 

In [22]:
# 可以使用 multidict() 更便捷的创建tuplelist和tupledict

cities, supply, demand = gp.multidict({
    'A': [100, 20],
    'B': [150, 50],
    'C': [20, 300],
    'D': [10, 200]
})

print("cities: {} \nsupply: {} \ndemand: {}".format(cities, supply, demand))
print("cities: {} \nsupply: {} \ndemand: {}".format(type(cities), type(supply), type(demand)))

cities: ['A', 'B', 'C', 'D'] 
supply: {'A': 100, 'B': 150, 'C': 20, 'D': 10} 
demand: {'A': 20, 'B': 50, 'C': 300, 'D': 200}
cities: <class 'gurobipy.tuplelist'> 
supply: <class 'gurobipy.tupledict'> 
demand: <class 'gurobipy.tupledict'>


In [58]:
# x 为一个 3*4 下标的变量(共12个)， 且其均为BINARY。 | x[0,1] x[0,2] .... x[2,3] -> 共12个
# 其中x为tupledict
# 产生如下限制条件
# x[0,0] + x[0,1] + x[0,2] + x[0,3] <= 1
# x[1,0] + x[1,1] + x[1,2] + x[1,3] <= 1
# x[2,0] + x[2,1] + x[2,2] + x[2,3] <= 1

m = gp.Model()
x = m.addVars(3, 4, vtype=GRB.BINARY, name='x')
m.addConstrs((x.sum(i, '*') <= 1 for i in range(3)), name='con')
m.update()
m.write("class1_example_0.lp")

In [ ]:
# Gurobi里面可以使用quicksum来代替sum，效率更高
# 以下两段代码是等价的
# x.prod(y) 相当于x与y下标相同的求积之后再求和
# 可以使用 x.prod(y, '*', 3)这样的来进行下标的筛选之后，再求积+求和

obj = gp.quicksum(cost[i,j]*x[i,j] for i,j in arcs if j == 3)
obj = x.prod(cost, '*', 3)

## 建模过程

按以下流程进行建模，不要混合在一起

1. Create all variables objects

    1.1 addVar()
    
    1.2 addVars()
   
   
2. Set Objective Function(s)

    2.1 setObjective()


3. Create Constraints

    3.1 addConstr()
    
    3.2 addConstrs()


4. Run Optimization

    4.1 optimize()

## Example1 - mip1

In [59]:
# Maximize
#   obj1: x + y + 2 z
# Subject To
#   c0: x + 2 y + 3 z <= 4
#   c1: x + y >= 1
# Binary
#   x y z
# End

In [61]:
# Import Env
import gurobipy as gp
from gurobipy import GRB

try:
    # Create a new model
    m = gp.Model('mip1')

    # Create variables
    x = m.addVar(vtype=GRB.BINARY, name='x')
    y = m.addVar(vtype=GRB.BINARY, name='y')
    z = m.addVar(vtype=GRB.BINARY, name='z')

    # Set Objective
    m.setObjective(x + y + 2 * z, GRB.MAXIMIZE)

    # Add constraint
    m.addConstr(x + 2 * y + 3 * z <= 4, 'c0')
    m.addConstr(x + y >= 1, 'c1')

    # Optimize Model
    m.optimize()

    # Print result
    for v in m.getVars():
        print('{} {}'.format(v.varName, v.x))
    print('Obj: {}'.format(m.objVal))

except gp.GurobiError as e:
    print('Error Code ' + str(e.errno) + ':' + str(e))

except AttributeError:
    print('Encountered an attribute error')

Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (mac64)
Optimize a model with 2 rows, 3 columns and 5 nonzeros
Model fingerprint: 0xb2adf8c4
Variable types: 0 continuous, 3 integer (3 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [1e+00, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+00]
Found heuristic solution: objective 2.0000000
Presolve removed 2 rows and 3 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 12 available processors)

Solution count 2: 3 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.000000000000e+00, best bound 3.000000000000e+00, gap 0.0000%
x 1.0
y 0.0
z 1.0
Obj: 3.0


## Example2 - Diet

人类需要四种营养. category: calories, protein, fat, sodium

食物来源. foods = hamburger, chicken, hot dog, fries, macaroni, pizza, salad, milk, ice cream

营养吸收每天有上下限，单位重量食物价格不同，单位重量食物所含营养成分不同

求达到足够营养的cost最小

In [80]:
# Import data

categories, minNutrition, maxNutrition = gp.multidict({
    'calories': [1800, 2000],
    'protein': [91, GRB.INFINITY],
    'fat': [0, 65],
    'sodium': [0, 1779]
})

foods, cost = gp.multidict({
    'hamburger': 2.49,
    'chicken': 2.89,
    'hot dog': 1.50,
    'fries': 1.89,
    'macaroni': 2.09,
    'pizza': 1.99,
    'salad': 2.49,
    'milk': 0.89,
    'ice cream': 1.59
})

nutritionValues = gp.tupledict({
    ('hamburger', 'calories'): 410,
    ('hamburger', 'protein'): 24,
    ('hamburger', 'fat'): 26,
    ('hamburger', 'sodium'): 730,
    ('chicken', 'calories'): 420,
    ('chicken', 'protein'): 32,
    ('chicken', 'fat'): 10,
    ('chicken', 'sodium'): 1190,
    ('hot dog', 'calories'): 560,
    ('hot dog', 'protein'): 20,
    ('hot dog', 'fat'): 32,
    ('hot dog', 'sodium'): 1800,
    ('fries', 'calories'): 380,
    ('fries', 'protein'): 4,
    ('fries', 'fat'): 19,
    ('fries', 'sodium'): 270,
    ('macaroni', 'calories'): 320,
    ('macaroni', 'protein'): 12,
    ('macaroni', 'fat'): 10,
    ('macaroni', 'sodium'): 930,
    ('pizza', 'calories'): 320,
    ('pizza', 'protein'): 15,
    ('pizza', 'fat'): 12,
    ('pizza', 'sodium'): 820,
    ('salad', 'calories'): 320,
    ('salad', 'protein'): 31,
    ('salad', 'fat'): 12,
    ('salad', 'sodium'): 1230,
    ('milk', 'calories'): 100,
    ('milk', 'protein'): 8,
    ('milk', 'fat'): 2.5,
    ('milk', 'sodium'): 125,
    ('ice cream', 'calories'): 330,
    ('ice cream', 'protein'): 8,
    ('ice cream', 'fat'): 10,
    ('ice cream', 'sodium'): 180
})

In [87]:
# Create Model
m = gp.Model('diet')

# Create Vars
# 创建下标(key)是食物，值(value)待定的tupledict
buyNum = m.addVars(foods, name='buyNum', vtype=GRB.INTEGER)
# buyNum = {}
# for f in foods:
#     buyNum[f] = m.addVar(name=f)


# Create Objective
m.setObjective(buyNum.prod(cost), GRB.MINIMIZE)
# m.setObjective(sum(buyNum[f] * cost[f] for f in foods), GRB.MINIMIZE)


# Create Constraints
# 来自于所有营养成分需要在其上下限之间
# 使用 '== [a, b]' 表示在[a,b]范围之内
# "_" 代表名字为前述循环的名字，在这个案例里为c
m.addConstrs(
    (gp.quicksum(nutritionValues[f, c] * buyNum[f] for f in foods) 
     == [minNutrition[c], maxNutrition[c]] 
     for c in categories), "_"
)
# m.addRange(expr, lhs, rhs, name)
# for c in categories:
#     m.addRange(
#         gp.quicksum(nutritionValues[f, c] * buyNum[f] for f in foods),
#         minNutrition[c], maxNutrition[c], c
#     )

def printSolution():
    '''
    :return: 优化结果
    '''
    if m.status == GRB.OPTIMAL:
        print('\nCost: %g' % m.objVal)
        print('\nBuy:')
        buyx = m.getAttr('x', buyNum)
        for f in foods:
            if buyNum[f].x > 0:
                print('%s %g' % (f, buyx[f]))
    else:
        print('No solution')
        
# Solve
m.optimize()
printSolution()

Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (mac64)
Optimize a model with 4 rows, 12 columns and 39 nonzeros
Model fingerprint: 0xaaaebbf6
Variable types: 3 continuous, 9 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [9e-01, 3e+00]
  Bounds range     [6e+01, 2e+03]
  RHS range        [6e+01, 2e+03]
Presolve removed 0 rows and 5 columns
Presolve time: 0.00s
Presolved: 4 rows, 7 columns, 25 nonzeros
Variable types: 0 continuous, 7 integer (1 binary)
Found heuristic solution: objective 12.7800000

Root relaxation: objective 1.183513e+01, 3 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   11.83513    0    3   12.78000   11.83513  7.39%     -    0s
     0     0   12.57000    0    3   12.78000   12.57000  1.64%     -    0s

Cutting planes:
  MIR: 2
  StrongCG: 1

Explored 1 nodes (6 simplex iteratio